In [1]:
# Task 3- cell 1
from google.colab import files

uploaded = files.upload()  # upload eval_ground_truth_counts.csv
gt_csv_path = list(uploaded.keys())[0]
print("Uploaded ground-truth file:", gt_csv_path)


NameError: name 'A' is not defined

In [ ]:
#cell 2
# Upload a zip file containing your frames (JPG/PNG)
uploaded = files.upload()  # choose e.g. JFK_4_frames.zip

zip_name = list(uploaded.keys())[0]
print("Uploaded:", zip_name)

# Unzip into /content/frames
os.makedirs("frames", exist_ok=True)
with zipfile.ZipFile(zip_name, 'r') as zip_ref:
    zip_ref.extractall("frames")

image_paths = sorted(glob("frames/**/*.*", recursive=True))
print(f"Found {len(image_paths)} images")
image_paths[:5]


In [ ]:
#Task 3 - cell 3
import os, shutil, numpy as np
from glob import glob

# Use the newly unzipped frames as source
all_frames = sorted(
    glob("frames/**/*.jpg", recursive=True) +
    glob("frames/**/*.png", recursive=True)
)

print("Total frames found in frames/:", len(all_frames))

num_eval = 50
indices = np.linspace(0, len(all_frames) - 1, num_eval, dtype=int)
eval_frames = [all_frames[i] for i in indices]

os.makedirs("eval_frames", exist_ok=True)

for p in eval_frames:
    shutil.copy(p, "eval_frames/" + os.path.basename(p))

print("Saved 50 evaluation frames to /content/eval_frames")


In [ ]:
# cell 4
!pip install ultralytics albumentations opencv-python-headless matplotlib tqdm --quiet

from ultralytics import YOLO
import albumentations as A
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from google.colab import files
import zipfile
from glob import glob
import json
import shutil
from tqdm import tqdm


In [ ]:
# cell 5
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


In [ ]:
# cell 6
model = YOLO("/content/drive/MyDrive/crowdhuman_models/best.pt")
print("Loaded fine-tuned model.")


In [ ]:
# cell 7
def yolo_heatmap_on_image(img_bgr, model, conf_thresh=0.15, sigma=25):
    """
    img_bgr: OpenCV BGR image
    model: YOLO model
    conf_thresh: minimum confidence for keeping a detection
    sigma: Gaussian blur strength for heatmap
    """
    h, w = img_bgr.shape[:2]

    # Run YOLO
    results = model.predict(
        source=img_bgr,
        conf=conf_thresh,
        verbose=False
    )[0]

    # Count persons + create point map
    point_map = np.zeros((h, w), dtype=np.float32)
    person_count = 0

    for box in results.boxes:
        cls = int(box.cls[0])
        conf = float(box.conf[0])
        if cls != 0 or conf < conf_thresh:  # class 0 = person
            continue

        x1, y1, x2, y2 = box.xyxy[0].tolist()
        cx = int((x1 + x2) / 2)
        cy = int((y1 + y2) / 2)

        if 0 <= cx < w and 0 <= cy < h:
            point_map[cy, cx] += 1.0
            person_count += 1

    # Turn point map into smooth heatmap
    heatmap = cv2.GaussianBlur(point_map, (0, 0), sigmaX=sigma, sigmaY=sigma)

    # Normalize heatmap to [0, 255]
    heatmap_norm = cv2.normalize(heatmap, None, 0, 255, cv2.NORM_MINMAX)
    heatmap_norm = heatmap_norm.astype(np.uint8)

    # Colorize
    heatmap_color = cv2.applyColorMap(heatmap_norm, cv2.COLORMAP_JET)

    # Overlay onto original frame
    overlay = cv2.addWeighted(img_bgr, 0.6, heatmap_color, 0.4, 0)

    # Get YOLO's built-in annotated image (boxes, labels)
    annotated = results.plot()  # returns BGR image

    return annotated, overlay, person_count


In [ ]:
# cell 8
pred_counts = {}  # recreate it from scratch

image_paths = sorted(glob("eval_frames/*.jpg") + glob("eval_frames/*.png"))

for img_path in image_paths:
    img_name = os.path.basename(img_path)
    img_bgr = cv2.imread(img_path)

    annotated, overlay, person_count = yolo_heatmap_on_image(
        img_bgr,
        model,
        conf_thresh=0.15,
        sigma=25
    )

    pred_counts[img_name] = person_count
    print(f"{img_name}: predicted {person_count} people")


In [ ]:
# cell 9
import csv
import numpy as np

# Load ground-truth counts
gt_counts = {}
with open(gt_csv_path, "r") as f:
    reader = csv.DictReader(f)
    for row in reader:
        gt_counts[row["frame"]] = int(row["true_count"])

# Compute per-frame absolute errors on frames that appear in both
errors = []
for name, true_c in gt_counts.items():
    if name in pred_counts:
        err = abs(true_c - pred_counts[name])
        errors.append(err)
        print(f"{name}: true={true_c}, pred={pred_counts[name]}, |err|={err}")

mae = float(np.mean(errors)) if errors else None
print("\nNumber of eval frames used:", len(errors))
print("Mean Absolute Error (MAE):", mae)
